In [8]:
import pdfplumber
import pandas as pd
import re
import pytesseract
import pdf2image

In [9]:
def is_text_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            first_page = pdf.pages[0]
            text = first_page.extract_text()
            return bool(text.strip())  # Check if there is any text
    except Exception as e:
        print(f"Error in detecting text-based PDF: {e}")
        return False


def convert_pdf_to_images(pdf_path):
    from pdf2image import convert_from_path

    images = convert_from_path(pdf_path)
    return images


def extract_image_pdf(pdf_path):
    extracted_text = []
    images = convert_pdf_to_images(pdf_path)

    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_text.append(text)

    return extracted_text

In [10]:
file = r"D:\OneDrive - Kemenkeu\PEMERIKSA\Job Shadowing\2.Pembimbingan Materi Pemeriksaan\Cahaya Triagro Soy\Data WP\Rekening Koran\BNI 2022\01-03-2022.PDF"
text = extract_image_pdf(file)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [4]:
def extract_text_pdf(pdf_path):
    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                extracted_text.append(text)
    return extracted_text

In [5]:
text = extract_text_pdf(file)

In [7]:
text

[]

In [6]:
data = []
keterangan = []
for halaman, page in enumerate(text):
    lines = page.split("\n")

    current_entry = {}
    header_found = False
    header_line = 0

    for index, line in enumerate(lines):
        if (
            "TANGGAL" in line
            and "KETERANGAN" in line
            and "MUTASI" in line
            and "SALDO" in line
        ):
            header_found = True
            header_line = index

            break

    if header_found:
        # Try to match the date pattern at the beginning of the line

        line_data = lines[(header_line + 1) :]
        for index, line in enumerate(line_data):
            # for index, line in enumerate(lines[63:]):

            date_match = re.match(r"^(\d{2}/\d{2})(?!.*\*)", line)
            if date_match:
                current_entry = {"Tanggal": date_match.group(1)}
                parts = line.split()
                parts_cek = parts
                # parts_cek = [x.replace(",", "") for x in parts]
                # parts_cekpostif = [x.replace("-", "") for x in parts_cek]
                digits = re.findall(r"-?\d{1,3}(?:,\d{3})*\.\d{2}", line)
                count_digit = len(digits)

            #     if count_digit == 1 and parts_cek[1] == "SALDO":
            #         current_entry["SALDO"] = parts_cek[-1]
            #     elif count_digit == 1 and parts_cek[-1] == "DB":
            #         current_entry["MUTASI"] = parts_cek[-2]
            #         current_entry["DB/CR"] = parts_cek[-1]
            #     elif count_digit == 2 and "DB" in parts_cek:
            #         current_entry["DB/CR"] = "DB"
            #         current_entry["MUTASI"] = parts_cek[parts_cek.index(digits[0])]
            #         current_entry["SALDO"] = parts_cek[parts_cek.index(digits[1])]
            #     elif count_digit == 2 and "DB" not in parts_cek:
            #         current_entry["DB/CR"] = "CR"
            #         current_entry["MUTASI"] = parts_cek[parts_cek.index(digits[0])]
            #         current_entry["SALDO"] = parts_cek[parts_cek.index(digits[1])]
            #     elif count_digit == 1:
            #         current_entry["MUTASI"] = parts_cek[-1]
            #         current_entry["DB/CR"] = "CR"
            #     elif parts_cek[-1] == "DB":
            #         current_entry["DB/CR"] = parts_cek[-1]
            #         current_entry["MUTASI"] = parts_cek[-2]
            #     else:
            #         current_entry["DB/CR"] = "CR"
            #     edit_keterangan(data, keterangan)
            #     keterangan.clear()
            #     keterangan.append(" ".join(parts[1:-count_digit]))

            #     for col in range(count_digit):
            #         if parts[(len(parts) - count_digit) + col] == "DB":
            #             current_entry["3"] = parts[(len(parts) - count_digit) + col]
            #             current_entry["2"] = parts[
            #                 (len(parts) - count_digit) + col - 1
            #             ]

            #         elif count_digit == 1:
            #             continue
            #         else:
            #             current_entry[f"{col+1}"] = parts[
            #                 (len(parts) - count_digit) + col
            #             ]
            #     current_entry["halaman"] = halaman + 1
            #     data.append(current_entry)
            # else:
            #     if not line.strip().lower().startswith("bersambung"):
            #         keterangan.append(line.strip())

            # current_entry["Keterangan"] = ""

In [ ]:
page1 = text[0]
lines = page1.split("\n")
lines[21]

In [27]:
line = lines[22]

date_pattern = r"\b\d{2}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}\b"
teller_pattern = r"\b\d{7}\b"  # Match exactly 7 digits for teller number
uraian_pattern = (
    r"\b[a-zA-Z\s]+(?=\s\d{7}\s\d{1,3}(?:,\d{3})*\.\d{2}|\s\d{1,3}(?:,\d{3})*\.\d{2}\b)"
)
amount_pattern = r"\b\d{1,3}(?:,\d{3})*(?:\.\d{2})?\b"

date = re.search(date_pattern, line)
if date:
    date = date.group(0).strip()

    # Extracting teller
    teller = re.search(teller_pattern, line)
    if teller:
        teller = teller.group(0).strip()

    # Extracting uraian
    uraian = re.search(uraian_pattern, line)
    if uraian:
        uraian = uraian.group(0).strip()

    # Extracting amounts
    amounts = re.findall(amount_pattern, line)
    amounts = [amount for amount in amounts if not re.match(r"^\d{2}$", amount)]
print(f"date: {date}")
print(f"teller: {teller}")
print(f"uraian: {uraian}")
print(f"amounts: {amounts}")

date: None
teller: 8888145
uraian: NBMB DARMINI TO CAHAYA TRIAGRO SO
amounts: ['0.00', '130,000,000.00', '171,938,031.00']


In [ ]:
page1 = text[0]
lines = page1.split("\n")

current_entry = {}
header_found = False
header_line = 0

for index, line in enumerate(lines):
    # Identify the header row
    if (
        "Tanggal Transaksi" in line
        and "Uraian Transaksi" in line
        and "Teller " in line
        and "Debet" in line
        and "Kredit" in line
        and "Saldo" in line
    ):
        header_found = True
        header_line = index + 1

        break
if header_found:
    # Try to match the date pattern at the beginning of the line

    line_data = lines[(header_line + 1) :]
else:
    line_data = lines

for index, line in enumerate(line_data):
    date_pattern = r"\b\d{2}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}\b"
    teller_pattern = r"\b\d{7}\b"  # Match exactly 7 digits for teller number
    uraian_pattern = r"\b[a-zA-Z\s]+(?=\s\d{7}\s\d{1,3}(?:,\d{3})*\.\d{2}|\s\d{1,3}(?:,\d{3})*\.\d{2}\b)"
    amount_pattern = r"\b\d{1,3}(?:,\d{3})*(?:\.\d{2})?\b"

    date = re.search(date_pattern, line)
    if date is not None:
        date = date.group(0).strip()

        # Extracting teller
        teller = re.search(teller_pattern, line)
        if teller:
            teller = teller.group(0).strip()

        # Extracting uraian
        uraian = re.search(uraian_pattern, line)
        if uraian:
            uraian = uraian.group(0).strip()

        # Extracting amounts
        amounts = re.findall(amount_pattern, line)
        amounts = [amount for amount in amounts if not re.match(r"^\d{2}$", amount)]

        current_entry = {
            "tanggal transaksi": date,
            "uraian transaksi": uraian,
            "teller": teller,
            "debet": amounts[0],
            "kredit": amounts[1],
            "saldo": amounts[2],
        }
        data.append(current_entry)

01/12/22 12:47:42 NBMB DARMINI TO CAHAYA TRIAGRO SO 8888145 0.00 130,000,000.00 171,938,031.00
01/12/22 12:47:42
ESB:NBMB:0001500F:488693461279
02/12/22 12:04:42 NBMB DARMINI TO CAHAYA TRIAGRO SO 8888556 0.00 125,000,000.00 296,938,031.00
02/12/22 12:04:42
ESB:NBMB:0001500F:488989053825
02/12/22 12:49:58 BFST0943555777 NBMB:CENAIDJA 8888542 250,000,000.00 0.00 46,938,031.00
02/12/22 12:49:58
20221202BRINIDJA010O0230135683
ESB:NBMB:0008G00F:489001438694
02/12/22 12:49:58 BFST0943555777 NBMB:CENAIDJA 8888542 2,500.00 0.00 46,935,531.00
02/12/22 12:49:58
20221202BRINIDJA010O0230135683
ESB:NBMB:0008G00F:489001438694
03/12/22 17:55:36 NBMB ASEP ROHMANA TO CAHAYA TRIAGRO SO 8888133 0.00 83,357,000.00 130,292,531.00
03/12/22 17:55:36
ESB:NBMB:0001500F:489327780849
05/12/22 14:27:26 NBMB OIH RUKOYAH TO CAHAYA TRIAGRO SO 8888496 0.00 10,000,000.00 140,292,531.00
05/12/22 14:27:26
ESB:NBMB:0001500F:489754799610
05/12/22 17:18:57 QPGXCA67G WS_OB;c92ea19a7877cccf36d0;29496 0371893 0.00 100,000,000

In [ ]:
data = []
keterangan = []

teller_pattern = r"\b\d+\b"
account_number_pattern = r"\b\d{11}\b"
uraian_pattern = (
    r"\b[a-zA-Z\s]+(?=\s\d{7}\s\d{1,3}(?:,\d{3})*\.\d{2}|\s\d{1,3}(?:,\d{3})*\.\d{2}\b)"
)
date_pattern = r"\b\d{2}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}\b"
amount_pattern = r"\b\d{1,3}(?:,\d{3})*(?:\.\d{2})?\b"


def split_keterangan(keterangan):
    pattern = r"\s([CD])\s"
    parts = re.split(pattern, keterangan)
    parts = [part.strip() for part in parts if part.strip()]
    while len(parts) < 3:
        parts.append("")
    return parts


def bri_clean(text):
    for halaman, page in enumerate(text):
        lines = page.split("\n")

        current_entry = {}
        header_found = False
        header_line = 0

        for index, line in enumerate(lines):
            # Identify the header row
            if (
                "Tanggal Transaksi" in line
                and "Uraian Transaksi" in line
                and "Teller " in line
                and "Debet" in line
                and "Kredit" in line
                and "Saldo" in line
            ):
                header_found = True
                header_line = index + 1

                break
        if header_found:
            # Try to match the date pattern at the beginning of the line

            line_data = lines[(header_line + 1) :]
        else:
            line_data = lines

            for index, line in enumerate(line_data[:2]):
                name = re.search(date_pattern, line).group(0).strip()
                uraian = re.search(uraian_pattern, line).group(0)
                date = re.search(date_pattern, line).group(0).strip()
                amounts = re.findall(amount_pattern, line)

                # current_entry = {
                #     "nama": name,
                #     "nomor rekening": account_number,
                #     "tanggal transaksi": date,
                #     "keterangan": keterangan,
                #     "jenis transaksi": jenis,
                #     "remarks": remarks,
                #     "amounts": amounts,
                #     "saldo": saldo,
                # }
                # data.append(current_entry)